In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pprint

#### Note: we can't actually use pandas in our algorithm, but we can use it to read in our df

binary recursive partitioning. This is an iterative process of splitting the data into partitions, and then splitting it up further on each of the branches.

https://towardsdatascience.com/decision-tree-classification-de64fc4d5aac

https://towardsdatascience.com/what-is-a-decision-tree-22975f00f3e1

### Quantifying splits with impurity
https://victorzhou.com/blog/gini-impurity/

"Gini Impurity is the probability of incorrectly classifying a randomly chosen element in the dataset if it were randomly labeled according to the class distribution in the dataset."

Decision Tree Classifier
Using the decision algorithm, **we start at the tree root and split the data on the feature that results in the largest information gain (IG)** (reduction in uncertainty towards the final decision).
In an iterative process, we can then repeat this splitting procedure at each child node **until the leaves are pure**. This means that the samples at each leaf node all belong to the same class.
In practice, we may **set a limit on the depth of the tree** to prevent overfitting. We compromise on purity here somewhat as the final leaves may still have some impurity.


In [ ]:
import pandas as pd
df = pd.read_csv('Iris.csv')

In [ ]:
df.drop(columns='Id', inplace=True)

In [ ]:
df.head()

## Train/test split 

In [ ]:
def train_test_split(df, test_size):
    
    # get a list of indices
    indices = df.index.to_list()
    
    # if a float result, rounding test_size to an int is needed
    test_size = round(test_size * len(df))
    
    # get a random choice of the indices
    test_indices = np.random.default_rng().choice(indices, size=test_size)

    # create our test df
    test = df.loc[test_indices]
        
    # create our train df 
    train = df.drop(test_indices)
    
    # return both
    return train, test

In [ ]:
# see if percentages work 
train, test = train_test_split(df, 0.20)


In [ ]:
train.shape, test.shape

In [ ]:
# numpy array 
train_data = train.values
train_data[:5]

## Check purity

In [ ]:
def purity(data):
    
    # get our labels for this data 
    labels = data[:,-1]
    unique_classes = np.unique(labels)
    
    # see if we only have one label left
    if len(unique_classes) == 1:
        return True
    else: 
        return False

In [ ]:
# [np.unique(labels)[2]] * len(labels)

### Classify

In [ ]:
def classify(data):
    
    # get the true labels for the input data
    labels = data[:,-1]
    
    # get the unique labels and label counts for this input data
    classes, class_counts = np.unique(labels, return_counts=True)
    
    # get the majority class
    index = class_counts.argmax()
    
    # classify our input as the majority
    classification = classes[index]
    
    return classification

In [ ]:
def classify(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [ ]:
a = classify(train_data)

In [ ]:
classify(train[train.PetalWidthCm > 1.2].values)

### Potential splits

In [ ]:
train.tail()

In [ ]:
def get_potential_splits(data):
    
    potential_splits = {}

    # how many columns are there
    _, n_columns = data.shape
    
    # for each column in our array 
    for column_index in range(n_columns - 1):
    
        # we want our key to be the column index
        potential_splits[column_index] = []
    
        # we want our value to be each row value at that index
        values = data[:, column_index]
    
        # we want all the possible splits
        unique_values = np.unique(values)

        # for each index in range(15)
        for index in range(len(unique_values)):

            if index != 0:

                # get the sum of the current and previous values and 
                current_value = unique_values[index]
                previous_value = unique_values[index-1]
                potential_split = (current_value + previous_value) / 2

                potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [ ]:
potential_splits = get_potential_splits(train_data)

In [ ]:
# one for each column besides the label
potential_splits.keys()

In [ ]:
import seaborn as sns

In [ ]:
sns.lmplot(data=train, x='PetalWidthCm', y='PetalLengthCm', hue='Species', fit_reg=False)
plt.hlines(y=potential_splits[2], xmin=0, xmax=2.5);

### Split data

In [ ]:
train_data[:5]

In [ ]:
# which question is most important?
# ex. "is the petal width less/larger than value"
# we have four questions to ask it

# split dataset based on column and value
def split_data(data, split_col, split_val):
    
    # get that specific column's values
    split_column_values = data[:, split_col]
    
    # get the data below the specified value 
    data_below = data[split_column_values <= split_val]
    
    # get the data above the specified value
    data_above = data[split_column_values > split_val]
    
    return data_below, data_above

In [ ]:
split_col = 2
split_val = 2.45

In [ ]:
# Question: is our petal width < or > 0.8?
data_below, data_above = split_data(train_data, split_col, split_val)

Steps to calculate entropy for a split:

Calculate entropy of parent node
Calculate entropy of each individual node of split and calculate weighted average of all sub-nodes available in split.

entropy = sum(probs * -log2(probs))

In [51]:
 def calculate_entropy(data):
        
    labels = data[:,-1]
    _, counts = np.unique(labels, return_counts=True)

    probs = counts / counts.sum()

    entropy = sum(probs * -np.log2(probs))
    return entropy

In [52]:
def calculate_overall_entropy(data_below, data_above):
    
    total_data_points = len(data_below) + len(data_above)
    
    # weight of data below
    p_of_data_below = len(data_below) / total_data_points
    
    # weight of data above
    p_of_data_above = len(data_above) / total_data_points
    
    overall_entropy = (p_of_data_below * calculate_entropy(data_below))+(p_of_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [53]:
calculate_overall_entropy(data_below, data_above)

0.6430565257160958

In [ ]:
potential_splits = get_potential_splits(train_data)

In [ ]:
def find_best_split(data, potential_splits):
    
    '''Figure out where the best split is for given data'''
    
    # set entropy to abritrary high number
    overall_entropy = 500
    
    # for each dictionary key (0,1,2,3)
    for column_index in potential_splits:
        
        # for each value within that key
        for value in potential_splits[column_index]:
            
            # split the data 
            data_below, data_above = split_data(train_data, split_col=column_index, split_val=value)
            
            # calculate the entropy
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy < overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    
    
    return best_split_column, best_split_value

In [ ]:
find_best_split(train_data, potential_splits)

## Decision Tree Algorithm

In [ ]:
# must be a recursive function
def decision_tree(df, counter=0, min_samples=2, max_depth=5):
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values 
    else:
        data = df
    
    # base case
    if (purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify(data)
        return classification
    
    # if data not pure
    else:
        # increment counter
        counter += 1
        # recursion
        potential_splits = get_potential_splits(data)
        # Question: is our petal width < or > 0.8?
    
        split_col, split_val = find_best_split(data, potential_splits)
        
        data_below, data_above = split_data(data, split_col, split_val)
        
        feature_name = COLUMN_HEADERS[split_col]
        question = "{} <= {}".format(feature_name, split_val)
        subtree = {question: []}
        
        yes_answer = decision_tree(data_below, counter, min_samples, max_depth) 
        no_answer = decision_tree(data_above, counter, min_samples, max_depth)
        
        if yes_answer == no_answer:
            subtree = yes_answer
        else:
        
            subtree[question].append(yes_answer)
            subtree[question].append(no_answer)
        
        return subtree

In [ ]:
tree = decision_tree(train, max_depth=2)
pprint.pprint(tree)

## Classification

In [36]:
tree

{'PetalLengthCm <= 2.45': ['Iris-setosa', 'Iris-versicolor']}

In [ ]:
def classify_example(example, tree):
    
    # find the current question
    question = list(tree.keys())[0]
    # split it into the feature and value
    feature_name, operator, value = question.split()
    
    # our tree is made up on subtrees
    if example[feature_name] <= float(value):
        # yes_answer = class "Iris-setosa"
        answer = tree[question][0]
    else:
        # no answer = subtree
        answer = tree[question][1]
    
    if not isinstance(answer, dict):
            # return class
        return answer
    else:
        # classify subtree
        return classify_example(example, answer)

In [ ]:
classify_example(example, tree)

## Accuracy 

In [ ]:
import pandas as pd

In [ ]:
def calculate_accuracy(df, tree):
    
    # new column for our predictions
    df['predictions'] = df.apply(classify_example, axis=1, args=(tree,))
    #calculate how acc
    df['correct_predictions'] = df.predictions == df.Species
    
    accuracy = df['correct_predictions'].mean()
    
    return accuracy

In [ ]:
calculate_accuracy(train, tree)

In [ ]:
test

In [ ]:
test.iloc[12]

In [ ]:
# see if percentages work 
train, test = train_test_split(df, 0.20)
tree = decision_tree(train, max_depth=5)
acc = calculate_accuracy(test, tree)

In [ ]:
pprint.pprint(tree)

In [ ]:
acc

---------